In [ ]:
import os
import requests
import pandas as pd
import time
import base64
import re

# 百度云配置（需要自行申请）
API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXX'
SECRET_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
ACCESS_TOKEN_URL = 'XXXXXXXXXXXXXXXX'
DETECT_API_URL = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

def natural_sort_key(s):
    """自然排序函数：按数字顺序排列文件名"""
    return [int(text) if text.isdigit() else text.lower() 
            for text in re.split(r'(\d+)', s)]

def get_access_token():
    """获取百度云访问令牌"""
    params = {
        'grant_type': 'client_credentials',
        'client_id': API_KEY,
        'client_secret': SECRET_KEY
    }
    response = requests.get(ACCESS_TOKEN_URL, params=params)
    return response.json().get('access_token')

def detect_objects(image_path, access_token):
    """调用百度云多主体检测API"""
    try:
        with open(image_path, 'rb') as f:
            img_data = base64.b64encode(f.read()).decode()
        
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        params = {'access_token': access_token}
        data = {'image': img_data}
        
        response = requests.post(DETECT_API_URL, headers=headers, params=params, data=data)
        return response.json()
    except Exception as e:
        print(f"检测失败：{str(e)}")
        return None

def process_images():
    # 路径配置
    image_dir = r'XXXXXXXXXXXXXXXXXXXXXXX'
    output_dir = r'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    os.makedirs(output_dir, exist_ok=True)
    
    # 获取访问令牌
    access_token = get_access_token()
    if not access_token:
        print("无法获取访问令牌")
        return
    
    # 获取并排序图片文件
    image_files = [f for f in os.listdir(image_dir) 
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    image_files.sort(key=natural_sort_key)
    
    results = []
    total_count = len(image_files)
    
    # 遍历图片目录
    for idx, filename in enumerate(image_files, start=1):
        image_path = os.path.join(image_dir, filename)
        print(f"正在处理 [{idx}/{total_count}]: {filename}")
        
        # 调用API检测
        detection = detect_objects(image_path, access_token)
        
        # 解析结果
        if detection and 'result' in detection:
            objects = []
            for obj in detection['result']:
                obj_info = f"{obj['keyword']}(置信度:{obj['score']:.2f})"
                objects.append(obj_info)
            
            results.append({
                '序号': idx,
                '文件名': filename,
                '检测结果': '\n'.join(objects),
                '物体数量': len(objects)
            })
        else:
            results.append({
                '序号': idx,
                '文件名': filename,
                '检测结果': '检测失败',
                '物体数量': 0
            })
        
        time.sleep(0.5)  # 避免请求过频
    
    # 保存到Excel
    df = pd.DataFrame(results)
    output_path = os.path.join(output_dir, f'检测结果_{time.strftime("%Y%m%d%H%M")}.xlsx')
    df.to_excel(output_path, index=False)
    print(f"结果已保存至：{output_path}")

if __name__ == '__main__':
    process_images()